## Testing the implementation of ResLogit

We reproduce in this notebook the results of the paper ResLogit: A residual neural network logit model for data-driven choice modelling [1].

Inspired by https://github.com/LiTrans/reslogit/blob/master/main.ipynb.

In [ ]:
import os

# Remove/Add GPU use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


import sys

sys.path.append("../../")

import timeit
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import normalize

from choice_learn.models import reslogit_bis



In [ ]:
# Config
choices = ["low", "high"]
batch_size = 64 # fixed
learning_rate = 0.001
n_layers = 16
n_epochs = 1000
patience = 30
patience_increase = 3
improvement_threshold = 0.995

We follow the same data preparation as in the original paper in order to get the exact same results.

In [ ]:
#read data
raw_data = pd.read_csv("../../choice_learn/datasets/data/Pedestrian_Waittime.csv")


In [ ]:
# Sefine the inputs and output
x_data = raw_data.iloc[:,5:21]
y_data=raw_data["category"]

# List of explanatory variable names
variables = list(x_data.columns)

# Number of observations, variables and choices
n_obs = raw_data.shape[0]
n_vars = x_data.shape[1]
n_choices = len(choices)

# Slicing index for train and valid split
slice = np.floor(0.7 * n_obs).astype(int)

# Slices x and y datasets into train and valid datasets based on slice
train_x_data, valid_x_data = x_data.iloc[:slice], x_data.iloc[slice:]
train_y_data, valid_y_data = y_data.iloc[:slice], y_data.iloc[slice:]

# Number of train and valid batches
n_train_batches = train_y_data.shape[0] // batch_size
n_valid_batches = valid_y_data.shape[0] // batch_size

# Define ordinal levels for train and valid dataset
train_level_data = pd.DataFrame(columns = ["level1"])
for i in range (train_y_data.shape[0]):
    label_train = y_data[i]
    classifier_train = [1] * (label_train -1) + [0] * (n_choices - label_train)
    train_level_data_lenght = len(train_level_data)
    train_level_data.loc[train_level_data_lenght] = classifier_train

valid_level_data =pd.DataFrame(columns = ["level1"])
for i in range (valid_y_data.shape[0]):
    label_valid = y_data[i + train_y_data.shape[0]]
    classifier_valid = [1] * (label_valid -1) + [0] * (n_choices - label_valid)
    valid_level_data_length = len(valid_level_data)
    valid_level_data.loc[valid_level_data_length] = classifier_valid

In [ ]:
# Convert to TensorFlow tensor
train_x_tensor = tf.convert_to_tensor(train_x_data.values, dtype=tf.float64)
train_y_tensor = tf.convert_to_tensor(train_y_data.values , dtype = tf.int64)
valid_x_tensor = tf.convert_to_tensor(valid_x_data.values , dtype = tf.float64)
valid_y_tensor = tf.convert_to_tensor(valid_y_data.values , dtype = tf.int64)
train_level_tensor = tf.convert_to_tensor(train_level_data.values , dtype = tf.int64)
valid_level_tensor = tf.convert_to_tensor(valid_level_data.values , dtype = tf.int64)

In [ ]:
# Create ResNet model
training_object = reslogit_bis.TrainingResLogit()
training_object.main_model_reslogit(x=train_x_tensor, y=train_y_tensor, n_vars=n_vars, n_choices=n_choices, n_layers=n_layers)

### References
[1] ResLogit: A residual neural network logit model for data-driven choice modelling, Wong, M.; Farooq, B (2021), Transportation Research Part C: Emerging Technologies 126\
(URL: https://doi.org/10.1016/j.trc.2021.103050)

In [ ]:
#training loop
epoch = 0
valid_freq = min(200, n_train_batches)
best_validation_ll = np.inf
start_time = timeit.default_timer()
done_looping = False
step = 0


filename = "{}{}_bestmodel.pkl".format("reslogit", n_layers)
training_frame = pd.DataFrame(columns=["epoch", "minibatch", "batches", "train_ll", "valid_ll", "valid_err"])

while (epoch < n_epochs) and (not done_looping):
    epoch = epoch + 1
    training_ll = 0

    # Minibatch loop
    for i in range(n_train_batches):
        inputs = train_x_tensor[i * batch_size : (i+1) * batch_size]
        choice = train_y_tensor[i * batch_size : (i+1) * batch_size]-1

        minibatch_ll = training_object.train_model(inputs, choice).item()
        training_ll = (training_ll * i + minibatch_ll)/(i + 1)

        iteration = (epoch - 1) * n_train_batches + i

        # If the current iter has reached the valid_freq then evaluate the validation loss on the validation batches
        if (iteration + 1) % valid_freq == 0:
            validation_ll = training_object.validate_model(
                valid_x_tensor,(valid_y_tensor-1)).item()

            #check prediction accuracy
            error = training_object.error(valid_y_tensor).item()

            #################################
            # track and save training stats #
            #################################
            training_step = {
                "epoch": epoch,
                "minibatch": i + 1,
                "batches": n_train_batches,
                "train_ll": training_ll * n_train_batches,
                "valid_ll": validation_ll,
                "valid_err": error,
            }
            training_frame.loc[step] = training_step
            #################################

            if validation_ll < best_validation_ll:
                print(("epoch {:d}, minibatch {:d}/{:d}, "
                       "validation likelihood {:.2f}").format(
                        epoch, i + 1, n_train_batches, validation_ll))

                # improve patience if loss improvement is good enough
                if validation_ll < best_validation_ll * improvement_threshold:
                    patience = max(patience, iteration * patience_increase)

                # set the best loss to the new current (good) validation loss
                best_validation_ll = validation_ll

                error = training_object.error(valid_y_tensor).item()
                training_frame.loc[step, "valid_err"] = error
                print("validation error  {:.2%}".format(error))

                # save the best model
                # with open(filename, 'wb') as f:
                #     pickle.dump([training_object, config], f)

            step = step + 1

        if epoch > 200:
            done_looping = True
            break

end_time = timeit.default_timer()
run_time = end_time - start_time
print(run_time)

In [ ]:
# Analyze the accuracy of model
y_pred_valid = training_object.predict_validate(valid_x_tensor)

# Count the number alternatives
num_low = tf.reduce_sum(y_pred_valid == 1)
num_high = tf.reduce_sum(y_pred_valid == 2)

print("Confusion_matrix for validation data:")
print(confusion_matrix(valid_y_tensor,y_pred_valid))
print(classification_report(valid_y_tensor,y_pred_valid))